In [44]:
import sys
from scipy import ndimage as ndi
import pandas as pd
sys.path.append("C:/Users/sanik/Anaconda3/envs/brainlight/Lib/site-packages")
from brainlit.utils.ngl_pipeline import NeuroglancerSession

In [51]:
url = "s3://mouse-light-viz/precomputed_volumes/brain1"
ngl_skel = NeuroglancerSession(url+"_segments", mip=1)
working_ids = []
for seg_id in range(300):
    try:
        segment = ngl_skel.cv.skeleton.get(seg_id)
        working_ids.append(seg_id)
    except:
          pass

print (working_ids)

Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 24.29it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 45.58it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 52.04it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18.19it/s]


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 33.20it/s]

[2, 7, 11, 12, 13, 14, 20, 24, 28, 35, 40, 45, 48, 49, 51, 53, 59, 60, 65, 67, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 85, 86, 91, 97, 98, 99, 101, 103, 106, 107, 109, 111, 112, 115, 118, 120, 124, 126, 127, 129, 132, 133, 136, 137, 140, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 205, 207, 208, 214, 223, 224, 225, 227, 228, 229, 230, 231, 232, 234, 235, 236, 237, 240, 243, 246, 247, 248, 249, 250, 252, 253, 254, 255, 256, 257, 258, 259, 261, 262, 265, 266, 267, 269, 270, 271, 272, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 291, 293, 294, 295, 296, 298]


In [61]:
output_feats = 'C:/Users/sanik/Anaconda3/envs/brainlight/Data/logr_features.csv'
training = pd.DataFrame(columns=['File No.','Label','Gaussian', 'Gaussian Gradient', 'Gaussian Laplacian','Mid-Voxel'])
flag = 'w';
ids = working_ids
print (working_ids)

[2, 7, 11, 12, 13, 14, 20, 24, 28, 35, 40, 45, 48, 49, 51, 53, 59, 60, 65, 67, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 85, 86, 91, 97, 98, 99, 101, 103, 106, 107, 109, 111, 112, 115, 118, 120, 124, 126, 127, 129, 132, 133, 136, 137, 140, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 205, 207, 208, 214, 223, 224, 225, 227, 228, 229, 230, 231, 232, 234, 235, 236, 237, 240, 243, 246, 247, 248, 249, 250, 252, 253, 254, 255, 256, 257, 258, 259, 261, 262, 265, 266, 267, 269, 270, 271, 272, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 291, 293, 294, 295, 296, 298]


In [63]:
with open(output_feats,flag) as f:
    for seg_id in ids:
        segment = ngl_skel.cv.skeleton.get(seg_id)
        for v_id,vertex in enumerate(segment.vertices):
            ngl = NeuroglancerSession(url,mip=1)
            try:
                img, bounds, voxel = ngl.pull_voxel(7,v_id,nx=0,ny =0,nz = 0)
                print(bounds)
            except:
                pass

            g = ndi.gaussian_filter(img,[1,1,0.3])
            gb = float(g[0,0,0])
            g = float(g[[voxel[0]],[voxel[1]],[voxel[2]]])

            gg = ndi.gaussian_gradient_magnitude(img,[1,1,0.3])
            ggb = float(gg[0,0,0])
            gg = float(gg[[voxel[0]],[voxel[1]],[voxel[2]]])

            gl = ndi.gaussian_laplace(img,[1,1,0.3])
            glb = float(gl[0,0,0])
            gl = float(gl[[voxel[0]],[voxel[1]],[voxel[2]]])

            training = training.append({'File No.': seg_id, 'Label': 1,
                    'Gaussian':g, 'Gaussian Gradient':gg, 'Gaussian Laplacian':gl,
                    'Mid-Voxel':voxel}, ignore_index=True) 

            training = training.append({'File No.': seg_id, 'Label': 0,
                    'Gaussian':gb, 'Gaussian Gradient':ggb, 'Gaussian Laplacian':glb,
                    'Mid-Voxel':voxel}, ignore_index=True) 


            training.to_csv(f,header=False)

Downloading:   0%|                                                                               | 0/1 [00:00<?, ?it/s]

Bbox([14520, 4050, 2756],[14586, 4100, 2808], dtype=int32)


Downloading:   0%|                                                                               | 0/1 [00:00<?, ?it/s]

Bbox([14520, 4050, 2756],[14586, 4100, 2808], dtype=int32)


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 35.02it/s]


...

Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 26.13it/s]


Bbox([12078, 5300, 2184],[12144, 5350, 2236], dtype=int32)


Downloading: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 23.32it/s]


Bbox([12078, 5300, 2184],[12144, 5350, 2236], dtype=int32)


PermissionError: [Errno 13] Permission denied